In [3]:
# ✅ Setup: import dependencies
import pandas as pd
import numpy as np
import re

In [5]:
# ✅ Helper: clean column names
def clean_column_name(col_name):
    col = col_name.strip().lower()
    col = re.sub(r'[^0-9a-zA-Z_]', '_', col)
    col = re.sub(r'__+', '_', col)
    return col

In [7]:
# ✅ Helper: summary of NaNs
def null_summary(df):
    nulls = df.isnull().sum()
    pct = df.isnull().mean() * 100
    return pd.DataFrame({
        'null_count': nulls,
        'null_pct': pct,
        'dtype': df.dtypes
    }).sort_values(by='null_pct', ascending=False)

In [9]:
# ✅ Main function: analyze
def analyze(df):
    original_columns = df.columns.tolist()
    cleaned_columns = [clean_column_name(c) for c in df.columns]
    df.columns = cleaned_columns

    report = null_summary(df)
    report['is_constant'] = [df[c].nunique() == 1 for c in df.columns]
    report['is_duplicate_column'] = df.T.duplicated().tolist()

    conversion_suggestions = []
    for col in df.select_dtypes(include=['object']).columns:
        sample = df[col].dropna().astype(str).head(10)
        if sample.str.fullmatch(r'\d+').all():
            conversion_suggestions.append('int')
        elif pd.to_datetime(sample, errors='coerce').notnull().all():
            conversion_suggestions.append('datetime')
        else:
            conversion_suggestions.append(None)

    for i, col in enumerate(df.select_dtypes(include=['object']).columns):
        report.loc[col, 'suggest_dtype'] = conversion_suggestions[i]

    return report

In [11]:
# ✅ Load a test file
# Replace with your filename
df = pd.read_csv("data/sample_dirty_data.csv")

# ✅ Run analysis
report = analyze(df)

# ✅ View results
report


C:\Users\a_lip\AppData\Local\Temp\ipykernel_94316\3124059764.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(sample, errors='coerce').notnull().all():


,null_count,null_pct,dtype,is_constant,is_duplicate_column,suggest_dtype
join_date,1,20.0,object,False,False,datetime
employee_id,0,0.0,int64,False,False,NaN
salary_,0,0.0,int64,False,False,NaN
age,0,0.0,int64,False,False,NaN
is_active,0,0.0,bool,False,False,NaN
department,0,0.0,object,True,False,None
duplicate_salary,0,0.0,int64,False,True,NaN


In [16]:
def clean(df, report):
    """
    Apply non-destructive cleaning suggestions based on the analyze report.
    Returns a cleaned copy of the DataFrame and a cleaning log.
    """
    df_clean = df.copy()
    cleaning_log = []

    for _, row in report.iterrows():
        col = row['cleaned_name']
        suggestion = row['fix_suggestion']

        if suggestion == 'convert to int':
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('Int64')
            cleaning_log.append(f"Converted column '{col}' to integer.")

        elif suggestion == 'convert to datetime':
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
            cleaning_log.append(f"Converted column '{col}' to datetime.")

        elif suggestion == 'handle missing values':
            if pd.api.types.is_numeric_dtype(df_clean[col]):
                fill_value = df_clean[col].median()
                df_clean[col].fillna(fill_value, inplace=True)
                cleaning_log.append(f"Filled missing values in numeric column '{col}' with median ({fill_value}).")
            else:
                fill_value = df_clean[col].mode().iloc[0] if not df_clean[col].mode().empty else "missing"
                df_clean[col].fillna(fill_value, inplace=True)
                cleaning_log.append(f"Filled missing values in non-numeric column '{col}' with mode ('{fill_value}').")

        elif suggestion == 'drop column' or suggestion == 'drop duplicate':
            df_clean.drop(columns=[col], inplace=True)
            cleaning_log.append(f"Dropped column '{col}' ({suggestion}).")

    return df_clean, cleaning_log